# Spark 101

In [1]:
import pyspark
import warnings
import pandas as pd
import numpy as np

from pyspark.sql.functions import col, concat, lit, when, round, avg

spark = pyspark.sql.SparkSession.builder.getOrCreate()
warnings.filterwarnings('ignore')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 16:52:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/25 16:52:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

In [2]:
np.random.seed(42)

prog_lang = pd.DataFrame(dict(index=np.arange(50), language=np.random.choice(list(['python','r', 'java', 'matlab', 'c++', 'scala', 'html', 'tex']), 50)))

In [3]:
prog_lang.head()

,index,language
0,0,html
1,1,matlab
2,2,c++
3,3,html
4,4,java


In [4]:
df = spark.createDataFrame(prog_lang)

- View the schema of the dataframe

In [5]:
df.printSchema()

root
 |-- index: long (nullable = true)
 |-- language: string (nullable = true)



- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [6]:
df.show(5)

+-----+--------+
|index|language|
+-----+--------+
|    0|    html|
|    1|  matlab|
|    2|     c++|
|    3|    html|
|    4|    java|
+-----+--------+
only showing top 5 rows



### 2. Load the `mpg` dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

    b. Transform the `trans` column so that it only contains either `manual` or `auto`.

In [7]:
from pydataset import data

df = spark.createDataFrame(data('mpg'))

In [8]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



a. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

In [9]:
a = 'The'
b = 'has'
c = 'a'
d = 'cylinder'
e = 'engine.'
s = ' '

df = df.withColumn('message', concat(lit(f'{a}{s}'), df.year, lit(f'{s}'), df.manufacturer, lit(f'{s}'), df.model, lit(f'{s}{b}{s}{c}{s}'), df.cyl, lit(f'{d}{s}{e}')))
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|             message|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999 audi a4 ...|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
only showing top 5 rows



For each vehicle.

    b. Transform the `trans` column so that it only contains either `manual` or `auto`.

In [10]:
df = df.withColumn('trans', when(df.trans.contains('auto'), 'auto').otherwise('manual'))

df.show()

+------------+------------------+-----+----+---+------+---+---+---+---+-------+--------------------+
|manufacturer|             model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|             message|
+------------+------------------+-----+----+---+------+---+---+---+---+-------+--------------------+
|        audi|                a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|The 1999 audi a4 ...|
|        audi|                a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|The 1999 audi a4 ...|
|        audi|                a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|The 2008 audi a4 ...|
|        audi|                a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|The 2008 audi a4 ...|
|        audi|                a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|The 1999 audi a4 ...|
|        audi|                a4|  2.8|1999|  6|manual|  f| 18| 26|  p|compact|The 1999 audi a4 ...|
|        audi|                a4|  3.1|2008|  6|  auto|  f| 18| 27|  p|compact|The 2008 aud

### 3. Load the `tips` dataset as a spark dataframe.

    a. What percentage of observations are smokers?  
    b. Create a column that contains the tip percentage  
    c. Calculate the average tip percentage for each combination of sex and smoker.  

In [11]:
df = spark.createDataFrame(data('tips'))
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



a. What percentage of observations are smokers?

In [21]:
(df.where(df.smoker == 'Yes').count()) / (df.count()) * 100

38.114754098360656

b. Create a column that contains the tip percentage.

In [13]:
df = df.withColumn('tip_percentage', round((col('tip')/col('total_bill')) * 100, 2))
df.show(5)

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|         16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|         16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|         13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|         14.68|
+----------+----+------+------+---+------+----+--------------+
only showing top 5 rows



c. Calculate the average tip percentage for each combination of sex and smoker.

In [22]:
result = df.groupBy('sex', 'smoker').agg(round(avg('tip_percentage'), 2).alias('avg_tip_percentage'))

result.show()

+------+------+------------------+
|   sex|smoker|avg_tip_percentage|
+------+------+------------------+
|  Male|    No|             16.07|
|Female|    No|             15.69|
|  Male|   Yes|             15.28|
|Female|   Yes|             18.21|
+------+------+------------------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    a. Convert the temperatures to fahrenheit.  
    b. Which month has the most rain, on average?    
    c. Which year was the windiest?  
    d. What is the most frequent type of weather in January?  
    e. What is the average high and low temperature on sunny days in July in 2013 and 2014?   
    f. What percentage of days were rainy in q3 of 2015?  
    g. For each year, find what percentage of days it rained (had non-zero precipitation).

In [23]:
from vega_datasets import data

df = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
df = spark.createDataFrame(df)
df.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



a. Convert the temperatures to fahrenheit. 

In [25]:
df = df.withColumn('temp_max', col('temp_max') * 1.8 + 32)
df = df.withColumn('temp_min', col('temp_min') * 1.8 + 32)

In [27]:
df.show()

+----------+-------------+------------------+------------------+----+-------+
|      date|precipitation|          temp_max|          temp_min|wind|weather|
+----------+-------------+------------------+------------------+----+-------+
|2012-01-01|          0.0|55.040000000000006|              41.0| 4.7|drizzle|
|2012-01-02|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05|          1.3|48.019999999999996|             37.04| 6.1|   rain|
|2012-01-06|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09|          4.3|             48.92|              41.0| 3.4|   rain|
|2012-01-10|          1.0|42.980000000000004|             33.08|